<a href="https://colab.research.google.com/github/anthemwingate/jiNx/blob/main/youtubePredictor_gptNeo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Initial Preparations**

1.   Add Dependencies
2.   Authenticate Google Cloud Storage
3.   Create local directory and Clone GPTNeo repo from Github
4.   Install requirements
5.   Declare local variables



In [ ]:
#@title Dependencies
!pip3 install --upgrade absl-py==0.11
!pip3 install --upgrade tensorflow-metadata

Requirement already up-to-date: absl-py==0.11 in /usr/local/lib/python3.7/dist-packages (0.11.0)
Requirement already up-to-date: tensorflow-metadata in /usr/local/lib/python3.7/dist-packages (0.29.0)


In [ ]:
#@title Setup
%tensorflow_version 2.x
!git clone https://github.com/EleutherAI/GPTNeo
%cd GPTNeo
!pip3 install -q -r requirements.txt
pretrained_model = None
dataset = None

fatal: destination path 'GPTNeo' already exists and is not an empty directory.
/content/GPTNeo


In [ ]:
from google.colab import auth
auth.authenticate_user()
!gcloud init

Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'
compute:
  gce_metadata_read_timeout_sec: '0'
core:
  account: anthemw@gmail.com

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  1

Your current configuration has been set to: [default]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

Choose the account you would like to use to perform operations for 
this configuration:
 [1] anthemw@gmail.com
 [2] Log in with a new account
Please enter your numeric choice:  1

You are logged in as: [anthemw@gmail.com].

Pick cloud project to use: 
 [1] single-patrol-30861

In [ ]:
path_to_cloud_bucket = 'gs://ditto_gptneo/' #@param {type:"string"}

**Set Up Dataset**


1.   Set Dataset YoutubeSubtitles and download data
2.   set dataset path variable, data 
3.   Set dataset name ytsub variable, ytsubs
4.   set out name variable, dataset name concatenated with tokenized



In [ ]:
import os
dataset = 'YoutubeSubtitles'
os.makedirs('data', exist_ok=True)
!wget https://the-eye.eu/public/AI/pile_preliminary_components/yt_subs.jsonl.zst -O data/yt_subs.jsonl.zst
dataset_path = 'data'
dataset_name = 'ytsubs'
out_name = dataset_name + "_tokenized"

--2021-03-27 17:15:05--  https://the-eye.eu/public/AI/pile_preliminary_components/yt_subs.jsonl.zst
Resolving the-eye.eu (the-eye.eu)... 162.213.130.242
Connecting to the-eye.eu (the-eye.eu)|162.213.130.242|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1782264700 (1.7G) [application/octet-stream]
Saving to: ‘data/yt_subs.jsonl.zst’

data/yt_subs.jsonl. 100%[===================>]   1.66G   112MB/s    in 16s     

2021-03-27 17:15:20 (108 MB/s) - ‘data/yt_subs.jsonl.zst’ saved [1782264700/1782264700]



**Tokenize Dataset and Upload to Google Cloud Storage**


1.   Create a file using dataset name variable, and write the dataset to that file in the dataset path variable
2.   Copy dataset to Google Cloud Storage



In [ ]:
# Tokenize Data
!python data/create_tfrecords.py --input_dir /content/GPTNeo/$dataset_path --name $dataset_name --files_per 1000 --output_dir $out_name --write_dataset_config --processes 1

# copy the data to your bucket
if not path_to_cloud_bucket.endswith('/'):
  path_to_cloud_bucket += '/'
copy_loc = path_to_cloud_bucket + "datasets/" + dataset
!gsutil -m cp -r /content/GPTNeo/$out_name $copy_loc
!gsutil ls $path_to_cloud_bucket

2021-03-27 17:19:36.102137: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Downloading: 100% 1.04M/1.04M [00:00<00:00, 5.82MB/s]
Downloading: 100% 456k/456k [00:00<00:00, 3.26MB/s]
Downloading: 100% 1.36M/1.36M [00:00<00:00, 7.31MB/s]
Writing TFRecord Files to ytsubs_tokenized/. Parsed 173619 input files. files_written : : 841it [2:44:19, 12.12s/it]{'discarded': 14935, 'processed': 173651, 'successful': 158716}
Writing TFRecord Files to ytsubs_tokenized/. Parsed 173619 input files. files_written : : 841it [2:44:20, 11.73s/it]
Copying file:///content/GPTNeo/ytsubs_tokenized/ytsubs_152_0_1000.tfrecords [Content-Type=application/octet-stream]...
Copying file:///content/GPTNeo/ytsubs_tokenized/ytsubs_385_0_1000.tfrecords [Content-Type=application/octet-stream]...
Copying file:///content/GPTNeo/ytsubs_tokenized/ytsubs_215_0_1000.tfrecords [Content-Type=application/octet-stream]...
Copying file:///content/GPTNeo/ytsubs_to

**Set Model Configs**

In [ ]:
%%writefile configs/dataset_configs/ytsubs.json

{
  "path": "gs://ditto_gptneo/datasets/dataset_configs/ytsubs*.tfrecords",
  "eval_path": "",
  "n_vocab": 50256,
  "tokenizer_is_pretrained": true,
  "tokenizer_path": "gpt2",
  "eos_id": 50256,
  "padding_id": 50257
}

Writing configs/dataset_configs/ytsubs.json


In [ ]:
%%writefile configs/GPT3_XL.json

{
    "n_head": 16,
    "n_vocab": 50257,
    "embed_dropout": 0,
    "lr": 0.0002,
    "lr_decay": "cosine",
    "warmup_steps": 3000,
    "beta1": 0.9,
    "beta2": 0.95,
    "epsilon": 1e-8,
    "opt_name": "adam",
    "weight_decay": 0,
    "train_batch_size": 256,
    "attn_dropout": 0,
    "train_steps": 600000,
    "eval_steps": 0,
    "predict_steps": 1,
    "res_dropout": 0,
    "eval_batch_size": 4,
    "predict_batch_size": 1,
    "iterations": 100,
    "n_embd": 2048,
    "datasets": [["pile", null, null, null]],
    "model": "GPT",
    "model_path": "gs://ditto_gptneo/GPT3_XL",
    "n_ctx": 2048,
    "n_layer": 24,
    "scale_by_depth": true,
    "scale_by_in": false,
    "attention_types" :  [[["global", "local"],12]],
    "mesh_shape": "x:4,y:2",
    "layout": "intermediate_expanded:x,heads:x,vocab:n_vocab,memory_length:y,embd:y",
    "activation_function": "gelu",
    "recompute_grad": true,
    "gradient_clipping": 1.0,
    "tokens_per_mb_per_replica": 2048,
    "precision": "bfloat16"
}

Writing configs/GPT3_XL.json


**Pretrained Model**


1.   Download the pretrained model weights
2.   Set path to local weight variable
3.   Upload the model to Google Cloud Storeage



In [ ]:
# @title Download pretrained model weights:
pretrained_model = 'GPT3_XL' #@param ["GPT3_XL", "GPT3_1-3B"]
!wget -m -np -c -U "eye02" -w 2 -R "index.html*" "https://the-eye.eu/public/AI/gptneo-release/GPT3_XL/"
path_to_local_weights = f"/content/GPTNeo/the-eye.eu/public/AI/gptneo-release/{pretrained_model}"

--2021-03-27 20:08:28--  https://the-eye.eu/public/AI/gptneo-release/GPT3_XL/
Resolving the-eye.eu (the-eye.eu)... 162.213.130.242
Connecting to the-eye.eu (the-eye.eu)|162.213.130.242|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘the-eye.eu/public/AI/gptneo-release/GPT3_XL/index.html.tmp’

the-eye.eu/public/A     [ <=>                ]  10.35K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2021-03-27 20:08:29 (77.6 MB/s) - ‘the-eye.eu/public/AI/gptneo-release/GPT3_XL/index.html.tmp’ saved [10598]

Loading robots.txt; please ignore errors.
--2021-03-27 20:08:31--  https://the-eye.eu/robots.txt
Reusing existing connection to the-eye.eu:443.
HTTP request sent, awaiting response... 200 OK
Length: 4 [text/plain]
Saving to: ‘the-eye.eu/robots.txt’

the-eye.eu/robots.t 100%[===================>]       4  --.-KB/s    in 0s      

2021-03-27 20:08:31 (1.27 MB/s) - ‘the-eye.eu/robots.txt’ saved [4

In [ ]:
# upload to your bucket
bucket_base = "gs://" + path_to_cloud_bucket.replace('gs://', '').split('/')[0]
!gsutil -m cp -r $path_to_local_weights $bucket_base

Copying file:///content/GPTNeo/the-eye.eu/public/AI/gptneo-release/GPT3_XL/model.ckpt-362000.data-00011-of-00032 [Content-Type=application/octet-stream]...
Copying file:///content/GPTNeo/the-eye.eu/public/AI/gptneo-release/GPT3_XL/model.ckpt-362000.data-00014-of-00032 [Content-Type=application/octet-stream]...
Copying file:///content/GPTNeo/the-eye.eu/public/AI/gptneo-release/GPT3_XL/model.ckpt-362000.data-00023-of-00032 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This 

In [ ]:
!gsutil ls $bucket_base

gs://ditto_gptneo/GPT3_XL/
gs://ditto_gptneo/datasets/


In [ ]:
# @title Modify config for colab. 
  
import json
from pprint import pprint

path_to_model = "gs://ditto_gptneo/GPT3_XL" #@param {type:"string"}
batch_size = 8 #@param {type:"integer"}
dset = "ytsubs"  #@param {type:"string"}
mesh_shape = "x:4,y:2" #@param {type:"string"}
train_steps = 1000 #@param {type:"integer"}
steps_per_checkpoint = 500 #@param {type:"integer"}
start_step = 400000 if pretrained_model == "GPT3_2-7B" else 362000

if path_to_model == "":
  path_to_model = f'{bucket_base.strip("/")}/{pretrained_model}'
print(f'MODEL PATH: {path_to_model}\n')

if dset == "" and dataset != "Sampling_Only":
  dset = dataset
elif dataset is None:
  dset = "pile"

def pad_to_multiple_of(n, mult):
  """
  pads n to a multiple of mult
  """
  extra = n % mult
  if extra > 0:
      n = n + mult - extra
  return n

with open(f'{path_to_local_weights}/config.json', 'r') as f:
  data = json.load(f)
  pprint(data)
  dset_val = [[dset, None, None, None]] if dset != "" else data["datasets"]
  mods = {
          "mesh_shape": mesh_shape,
          "layout": "intermediate_expanded:x,heads:x,memory_length:y,embd:y",
          "model_path": path_to_model,
          "datasets": dset_val,
          "train_steps": start_step + train_steps,
          "eval_steps": 0,
          "train_batch_size": batch_size,
          "predict_batch_size": batch_size
        }
  data.update(mods)
  print('\n--->\n')
  pprint(data)
  with open(f'configs/{pretrained_model}.json', 'w') as outfile:
    json.dump(data, outfile, indent=2)

MODEL PATH: gs://ditto_gptneo/GPT3_XL

{'activation_function': 'gelu',
 'attention_types': [[['global', 'local'], 12]],
 'attn_dropout': 0,
 'beta1': 0.9,
 'beta2': 0.95,
 'datasets': [['pile', None, None, None]],
 'embed_dropout': 0,
 'eos_id': 50256,
 'epsilon': 1e-08,
 'eval_batch_size': 128,
 'eval_steps': 10,
 'gradient_clipping': 1.0,
 'iterations': 500,
 'layout': 'batch:x,memory_length:y,embd:y',
 'lr': 0.0002,
 'lr_decay': 'cosine',
 'lr_decay_end': 300000,
 'mesh_shape': 'x:128,y:2',
 'model_path': 'gs://neo-d/models/GPT3_XL_Pile',
 'n_ctx': 2048,
 'n_embd': 2048,
 'n_head': 16,
 'n_layer': 24,
 'n_vocab': 50257,
 'opt_name': 'adam',
 'padding_id': 50257,
 'precision': 'bfloat16',
 'predict_batch_size': 128,
 'predict_steps': 0,
 'recompute_grad': True,
 'res_dropout': 0,
 'scale_by_depth': True,
 'scale_by_in': False,
 'tokens_per_mb_per_replica': 4096,
 'train_batch_size': 512,
 'train_steps': 400000,
 'warmup_steps': 3000,
 'weight_decay': 0}

--->

{'activation_function':

**Sample from the model**


1.   Add prompt file to local colab directory using colab file upload functionality in browser menu
2.   Execute Model Sampling



In [ ]:
#@title Add Prompt DiTTO_prompt and Execute

In [ ]:
%%writefile DiTTo_prompt.txt

class GPT(nn.Module):
    """  the full GPT language model, with a context size of block_size """

    def __init__(self, config):
        super().__init__()

        # input embedding stem
        self.tok_emb = nn.Embedding(config.vocab_size, config.n_embd)
        self.pos_emb = nn.Parameter(torch.zeros(1, config.block_size, config.n_embd))
        self.drop = nn.Dropout(config.embd_pdrop)
        # transformer
        self.blocks = nn.Sequential(*[Block(config) for _ in range(config.n_layer)])
        # decoder head
        self.ln_f = nn.LayerNorm(config.n_embd)
        self.head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        self.block_size = config.block_size
        self.apply(self._init_weights)

        logger.info("number of parameters: %e", sum(p.numel() for p in self.parameters()))

Writing DiTTo_prompt.txt


In [ ]:
!python3 main.py --model $pretrained_model --steps_per_checkpoint 500 --tpu colab --predict --prompt DiTTo_prompt.txt

2021-03-27 20:29:46.607048: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Instructions for updating:
non-resource variables are not supported in the long term
Current step 362000
Saving config to gs://ditto_gptneo/GPT3_XL
2021-03-27 20:30:04.934964: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-27 20:30:04.936865: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-03-27 20:30:04.950177: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-03-27 20:30:04.950235: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (b9f32c1e6fe8): /proc/driver/nvidia/version does not exist
2021-03-27 20:30:06.110147: I tensorflow/compiler/mlir/mlir_grap

**Evaluate the Model**

**Wikitext Dataset Evaluation**


1.   Download wikitext test set
2.   Tokenize and upload to Google Cloud Storage
3.   Create dataset config
4.   Update model config to point to wikitext test set
5.   Run the model in eval mode over tokenized data




In [ ]:
wikitext103_src = "https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip"
!wget $wikitext103_src
!unzip wikitext-103-raw-v1.zip

--2021-03-27 21:01:22--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.192.16
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.192.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191984949 (183M) [application/zip]
Saving to: ‘wikitext-103-raw-v1.zip’

wikitext-103-raw-v1 100%[===================>] 183.09M  72.6MB/s    in 2.5s    

2021-03-27 21:01:25 (72.6 MB/s) - ‘wikitext-103-raw-v1.zip’ saved [191984949/191984949]

Archive:  wikitext-103-raw-v1.zip
   creating: wikitext-103-raw/
  inflating: wikitext-103-raw/wiki.test.raw  
  inflating: wikitext-103-raw/wiki.valid.raw  
  inflating: wikitext-103-raw/wiki.train.raw  


In [ ]:

!mkdir wikitext
!mv /content/GPTNeo/wikitext-103-raw/wiki.test.raw wikitext/wikitext_test.txt

# Tokenize Data
!python data/create_tfrecords.py --input_dir wikitext --name wikitext --files_per 1000 --output_dir wikitext_tokenized --write_dataset_config --processes 1 --wikitext-detokenize

# copy the data to your bucket
if not path_to_cloud_bucket.endswith('/'):
  path_to_cloud_bucket += '/'
copy_loc = path_to_cloud_bucket 
!gsutil -m cp -r wikitext_tokenized $copy_loc
!gsutil ls $path_to_cloud_bucket

2021-03-27 21:02:17.757328: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Writing TFRecord Files to wikitext_tokenized/. Parsed 0 input files. files_written : 0it [00:02, ?it/s]
{'discarded': 0, 'processed': 1, 'successful': 1}
Copying file://wikitext_tokenized/wikitext_0_139.tfrecords [Content-Type=application/octet-stream]...
/ [1/1 files][578.6 KiB/578.6 KiB] 100% Done                                    
Operation completed over 1 objects/578.6 KiB.                                    
gs://ditto_gptneo/GPT3_XL/
gs://ditto_gptneo/datasets/
gs://ditto_gptneo/wikitext_tokenized/


In [ ]:
%%writefile configs/dataset_configs/wikitext.json

{
  "path": "",
  "eval_path": "gs://ditto_gptneo/wikitext_tokenized/*.tfrecords",
  "n_vocab": 50256,
  "tokenizer_is_pretrained": true,
  "tokenizer_path": "gpt2",
  "eos_id": 50256,
  "padding_id": 50257
}

Overwriting configs/dataset_configs/wikitext.json


In [ ]:
# @title Modify config for wikitext. 
  
import json
from pprint import pprint

batch_size = 8 #@param {type:"integer"}
assert pretrained_model is not None
with open(f'configs/{pretrained_model}.json', 'r') as f:
  data = json.load(f)
  pprint(data)
  dset_val = [["wikitext", None, None, None]]
  mods = {
          "datasets": dset_val,
          "eval_steps": 139 // batch_size,
          "train_batch_size": batch_size,
          "eval_batch_size": batch_size,
        }
  data.update(mods)
  print('\n--->\n')
  pprint(data)
  with open(f'{pretrained_model}.json', 'w') as outfile:
    json.dump(data, outfile, indent=2)

{'activation_function': 'gelu',
 'attention_types': [[['global', 'local'], 12]],
 'attn_dropout': 0,
 'beta1': 0.9,
 'beta2': 0.95,
 'datasets': [['wikitext', None, None, None]],
 'embed_dropout': 0,
 'eos_id': 50256,
 'epsilon': 1e-08,
 'eval_batch_size': 8,
 'eval_steps': 17,
 'eval_tasks': '',
 'gradient_clipping': 1.0,
 'iterations': 500,
 'layout': 'intermediate_expanded:x,heads:x,memory_length:y,embd:y',
 'lr': 0.0002,
 'lr_decay': 'cosine',
 'lr_decay_end': 300000,
 'mesh_shape': 'x:4,y:2',
 'model_path': 'gs://ditto_gptneo/GPT3_XL',
 'n_ctx': 2048,
 'n_embd': 2048,
 'n_head': 16,
 'n_layer': 24,
 'n_vocab': 50257,
 'opt_name': 'adam',
 'padding_id': 50257,
 'precision': 'bfloat16',
 'predict_batch_size': 8,
 'predict_steps': 0,
 'recompute_grad': True,
 'res_dropout': 0,
 'scale_by_depth': True,
 'scale_by_in': False,
 'tokens_per_mb_per_replica': 4096,
 'train_batch_size': 8,
 'train_steps': 363000,
 'warmup_steps': 3000,
 'weight_decay': 0}

--->

{'activation_function': 'gel

In [ ]:
!python3 main.py --eval --tpu colab --model $pretrained_model

2021-03-28 00:58:50.526720: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Instructions for updating:
non-resource variables are not supported in the long term
Current step 362000
Saving config to gs://ditto_gptneo/GPT3_XL
2021-03-28 00:58:56.167556: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-28 00:58:56.168650: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-03-28 00:58:56.179161: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-03-28 00:58:56.179214: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (b9f32c1e6fe8): /proc/driver/nvidia/version does not exist
2021-03-28 00:58:56.750935: I tensorflow/compiler/mlir/mlir_grap

**Lambada**

Modify config for Lambada and Run Lambada eval



In [ ]:
# @title Modify config for Lambada. 
  
import json
from pprint import pprint

batch_size = 8 #@param {type:"integer"}
assert pretrained_model is not None
with open(f'configs/{pretrained_model}.json', 'r') as f:
  data = json.load(f)
  mods = {
          "datasets": dset_val,
          "eval_steps": 0,
          "train_batch_size": batch_size,
          "eval_batch_size": batch_size,
          "eval_tasks": ["lambada"]
        }
  data.update(mods)
  print('\n--->\n')
  pprint(data)
  with open(f'configs/{pretrained_model}.json', 'w') as outfile:
    json.dump(data, outfile, indent=2)


--->

{'activation_function': 'gelu',
 'attention_types': [[['global', 'local'], 12]],
 'attn_dropout': 0,
 'beta1': 0.9,
 'beta2': 0.95,
 'datasets': [['wikitext', None, None, None]],
 'embed_dropout': 0,
 'eos_id': 50256,
 'epsilon': 1e-08,
 'eval_batch_size': 8,
 'eval_steps': 0,
 'eval_tasks': ['lambada'],
 'gradient_clipping': 1.0,
 'iterations': 500,
 'layout': 'intermediate_expanded:x,heads:x,memory_length:y,embd:y',
 'lr': 0.0002,
 'lr_decay': 'cosine',
 'lr_decay_end': 300000,
 'mesh_shape': 'x:4,y:2',
 'model_path': 'gs://ditto_gptneo/GPT3_XL',
 'n_ctx': 2048,
 'n_embd': 2048,
 'n_head': 16,
 'n_layer': 24,
 'n_vocab': 50257,
 'opt_name': 'adam',
 'padding_id': 50257,
 'precision': 'bfloat16',
 'predict_batch_size': 8,
 'predict_steps': 0,
 'recompute_grad': True,
 'res_dropout': 0,
 'scale_by_depth': True,
 'scale_by_in': False,
 'tokens_per_mb_per_replica': 4096,
 'train_batch_size': 8,
 'train_steps': 363000,
 'warmup_steps': 3000,
 'weight_decay': 0}


In [ ]:
!python3 main.py --eval --tpu colab --model $pretrained_model

2021-03-28 01:02:41.710327: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Instructions for updating:
non-resource variables are not supported in the long term
Current step 362000
Saving config to gs://ditto_gptneo/GPT3_XL
2021-03-28 01:02:47.309985: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-28 01:02:47.311196: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-03-28 01:02:47.322203: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-03-28 01:02:47.322266: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (b9f32c1e6fe8): /proc/driver/nvidia/version does not exist
2021-03-28 01:02:47.916609: I tensorflow/compiler/mlir/mlir_grap

**Deployment**

In [ ]:
!pip install urllib3==1.25.1
#!pip3 install watson_machine_learning_client

Reason for being yanked: Broken release
     |████████████████████████████████| 153kB 6.2MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: botocore 1.20.39 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.25.1 which is incompatible.
  Found existing installation: urllib3 1.26.4
    Uninstalling urllib3-1.26.4:
      Successfully uninstalled urllib3-1.26.4


In [ ]:
# import dependencies
from ibm_watson_machine_learning import APIClient

# Store access credentials
watson_ml_credentials = {
    "apikey": "q-BsYkp-xwvModz7CVPWKuuIxVCm8MEeMXrJFnrXQl1E",
    "url": "https://us-south.ml.cloud.ibm.com",
}

wml_client = APIClient(watson_ml_credentials)

# Specify model constants
MODEL_NAME = "GPT3_XL"
DEPLOYMENT_NAME = "youtubePredictor_gptNeo"
BEST_MODEL = pretrained_model

# Specify IBM Watson Machine Learning schema
model_properties = {
    wml_client.repository.ModelMetaNames.NAME: "{}".format(MODEL_NAME)
}

#Save Model
# @TODO add training_data, i.e. x values, from init.csv
# @TODO add training_target, i.e. y values from init.csv
# @TODO add pipeline, i.e. 
published_model_details = wml_client.repository.store_model(model=BEST_MODEL, meta_props=model_properties,training_data=,training_target=, pipeline=,)
model_uid = wml_client.repository.get_model_uid(model_details=published_model_details)
deployment = wml_client.deployments.create(artifact_uid=model_uid, name=DEPLOYMENT_NAME)

In [ ]:
!pip3 install git+https://github.com/huggingface/transformers.git

from transformers import GPT2Tokenizer, TFGPT2Model

text_filename = "prompt file absolute path goes here"
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
evaluation_model = TFGPT2Model.from_pretrained('gpt2')
generator = pipeline('question-answering', model=evaluation_model)
set_seed(42)
 generator("insert instruction from prompt", max_length=30, num_return_sequences=1)

with open(text_filename) as f:
    text = f.readlines()
 
encoded_input = tokenizer(text, return_tensors='tf')
output = evaluation_model(encoded_input)
output.save("/content/GPTNeo/models/gpt2/trained_gpt2")